In [42]:
import numpy as np
import sys
sys.path.append("..")
from reference.lib.envs.gridworld import GridworldEnv

In [44]:
env = GridworldEnv()

In [86]:
def policy_eval(policy, env, discount_factor=1.0, theta=0.00001):
    """
    Evaluate a policy given an environment and a full description of the environment's dynamics.
    
    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment. 
            env.nA is a number of actions in the environment.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: Gamma discount factor.
    
    Returns:
        Vector of length env.nS representing the value function.
    """
    # Start with a random (all 0) value function
    V = np.zeros(env.nS)
    
    # loop until convergence - system is stable
    while True:
        # this is the change value
        delta_value = 0
        
        # for every state value
        for i in range(len(V)):
            current_state = V[i]
            
            for policy_index in range(4):
                action = policy_index
                action_prob = policy[i,policy_index]
                
                for prob, next_state, reward, done in env.P[i][action]:
                    current_state = current_state + action_prob * prob * (reward + discount_factor * V[next_state]) 
                    
            delta_value = max(delta_value,np.abs(current_state-V[i]))
            V[i] = current_state
        if delta_value<theta: break
        print(V.reshape((4,4)))
        input()
    return np.array(V)

In [85]:
random_policy = np.ones([env.nS, env.nA]) / env.nA
v = policy_eval(random_policy, env)

print(v.reshape((4,4)))

[[ 0.         -1.         -1.25       -1.375     ]
 [-1.1875     -1.546875   -1.69921875 -1.91699219]
 [-1.55859375 -1.77636719 -1.86889648 -2.12878418]
 [-1.95257568 -2.16825867 -2.25109482  0.        ]]

[[ 0.         -2.94921875 -4.06835938 -4.70727539]
 [-3.64233398 -5.06365967 -5.92869568 -6.94413376]
 [-5.4392662  -6.41138744 -7.04888701 -7.65574378]
 [-7.36633891 -8.38393377 -8.23762916  0.        ]]

[[  0.          -6.9695282  -10.48682404 -12.77535725]
 [ -9.02258182 -13.14670795 -16.33533387 -19.73273575]
 [-15.11167906 -18.33418939 -20.68961106 -21.97106241]
 [-22.03162831 -25.41651758 -24.2416693    0.        ]]

[[  0.         -15.62029324 -25.29127613 -32.27999867]
 [-21.29647321 -32.04328038 -41.7745597  -51.7523248 ]
 [-39.34656473 -48.70818283 -55.86347962 -58.66479942]
 [-61.06624826 -70.95903783 -66.62418346   0.        ]]

[[   0.          -34.85900568  -59.84248618  -79.39870066]
 [ -50.11741811  -76.90807196 -103.86615034 -131.1977374 ]
 [-100.33690879 -125.72505

KeyboardInterrupt: 

In [22]:
# Test: Make sure the evaluated policy is what we expected
expected_v = np.array([0, -14, -20, -22, -14, -18, -20, -20, -20, -20, -18, -14, -22, -20, -14, 0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)

AssertionError: 
Arrays are not almost equal to 2 decimals

(mismatch 87.5%)
 x: array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.])
 y: array([  0, -14, -20, -22, -14, -18, -20, -20, -20, -20, -18, -14, -22,
       -20, -14,   0])

In [36]:
! start .